In [1]:
# Cell 1: Initial Setup and Configuration

import os
import pickle
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import logging

# Configure basic logging for feedback during execution.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration Constants ---

# Defines the root directory where the ADE20K dataset is stored.
# Assumes the notebook is in the root folder, and 'user-default-efs' is a subdirectory.
ADE20K_RAW_DATA_DIR = os.path.join('.', 'user-default-efs', 'ADE20K_2021_17_01')
logging.info(f"ADE20K raw data directory set to: {os.path.abspath(ADE20K_RAW_DATA_DIR)}")

# Specifies the path to the index file containing dataset metadata.
INDEX_FILE_PATH = os.path.join(ADE20K_RAW_DATA_DIR, 'index_ade20k.pkl')
logging.info(f"Index file path set to: {os.path.abspath(INDEX_FILE_PATH)}")

# Defines the root directory where the processed YOLOv8 compatible dataset will be stored.
YOLO_DATASET_ROOT_DIR = os.path.join('.', 'ADE20K_YOLOv8_Dataset')
logging.info(f"YOLOv8 dataset root directory set to: {os.path.abspath(YOLO_DATASET_ROOT_DIR)}")

# Defines subdirectories for images and labels within the YOLO dataset structure.
YOLO_IMAGES_DIR = os.path.join(YOLO_DATASET_ROOT_DIR, 'images')
YOLO_LABELS_DIR = os.path.join(YOLO_DATASET_ROOT_DIR, 'labels')

# Specifies the seed for random number generators to ensure reproducibility of dataset splits.
RANDOM_STATE_SEED = 42028
logging.info(f"Random state seed set to: {RANDOM_STATE_SEED}")

# Specifies the desired splits for training, validation, and testing sets.
# The test set size is explicitly defined, and the validation set is a proportion of the remaining data.
TRAIN_RATIO = 0.80  # 80% of the data for training.
VALIDATION_RATIO = 0.10  # 10% of the data for validation.
TEST_RATIO = 0.10  # 10% of the data for testing.
# Ensure ratios sum to 1.0 for clarity, though only two are strictly needed to define the three sets.
if not np.isclose(TRAIN_RATIO + VALIDATION_RATIO + TEST_RATIO, 1.0):
    raise ValueError("Train, validation, and test ratios must sum to 1.0.")
logging.info(f"Data split ratios: Train={TRAIN_RATIO}, Validation={VALIDATION_RATIO}, Test={TEST_RATIO}")

# Create the root directory for the YOLOv8 dataset if it does not already exist.
# This helps in organizing the processed data.
if not os.path.exists(YOLO_DATASET_ROOT_DIR):
    os.makedirs(YOLO_DATASET_ROOT_DIR)
    logging.info(f"Created YOLO dataset root directory: {os.path.abspath(YOLO_DATASET_ROOT_DIR)}")
else:
    logging.info(f"YOLO dataset root directory already exists: {os.path.abspath(YOLO_DATASET_ROOT_DIR)}")

# Create subdirectories for images and labels if they do not exist.
# These directories will store the actual image files and their corresponding annotation files.
for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(YOLO_IMAGES_DIR, subset), exist_ok=True)
    os.makedirs(os.path.join(YOLO_LABELS_DIR, subset), exist_ok=True)
logging.info("Ensured creation of train, val, and test subdirectories for images and labels.")

logging.info("Cell 1 execution completed: Initial setup and configuration are finished.")

2025-05-13 01:28:49,538 - INFO - ADE20K raw data directory set to: /home/sagemaker-user/user-default-efs/ADE20K_2021_17_01
2025-05-13 01:28:49,539 - INFO - Index file path set to: /home/sagemaker-user/user-default-efs/ADE20K_2021_17_01/index_ade20k.pkl
2025-05-13 01:28:49,539 - INFO - YOLOv8 dataset root directory set to: /home/sagemaker-user/ADE20K_YOLOv8_Dataset
2025-05-13 01:28:49,541 - INFO - Random state seed set to: 42028
2025-05-13 01:28:49,541 - INFO - Data split ratios: Train=0.8, Validation=0.1, Test=0.1
2025-05-13 01:28:49,542 - INFO - YOLO dataset root directory already exists: /home/sagemaker-user/ADE20K_YOLOv8_Dataset
2025-05-13 01:28:49,543 - INFO - Ensured creation of train, val, and test subdirectories for images and labels.
2025-05-13 01:28:49,544 - INFO - Cell 1 execution completed: Initial setup and configuration are finished.


In [ ]:
PROJECT_DIR = './YOLOv8_Training_Runs_ADE20K'
RUN_NAME = 'ade20k_yolov8s_seg_run12'

In [3]:
# Cell 2: Load Metadata and Define Curated Target Classes

import pickle
import numpy as np
import logging

# --- Define the Curated List of Target Object Classes ---
# This list contains common objects deemed relevant for the application.
# It is important to verify these names against the dataset's actual object names.
# Some names in ADE20K can be compound (e.g., "foo, bar, baz").
# The names below are chosen for their general applicability.

CURATED_OBJECT_CLASS_CANDIDATES = [
    # People & Accessories
    'person', 'backpack', 'handbag', 'suitcase', 'umbrella',
    # Vehicles
    'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'boat', 'airplane',
    # Traffic & Street Furniture
    'traffic light', 'stop sign', 'fire hydrant', 'parking meter', 'bench', 'street sign',
    # Animals (Common urban/domestic)
    'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', # Adjusted based on common dataset animals
    # Indoor Furniture
    'chair', 'sofa', 'potted plant', 'bed', 'dining table', 'table', 'desk', 'toilet', 'door', 'window',
    'bookshelf', 'cabinet',
    # Electronics
    'television', 'tv', 'monitor', 'laptop', 'mouse', 'remote control', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'blender',
    # Common Objects
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'bottle', 'cup', 'fork', 'knife',
    'spoon', 'bowl', 'plate',
    # Sports Equipment (Common)
    'tennis racket', 'baseball bat', 'sports ball', 'skateboard',
    # Environmental / Structural (Important for scene context)
    'building', 'house', 'sky', 'tree', 'road', 'sidewalk', 'wall', 'floor', 'ceiling', 'stairs', 'pole', 'fence', 'signboard',
    'light', 'light source', 'cushion', 'pillow'
    # Total: 79 candidates. This list can be trimmed or expanded.
]
logging.info(f"Defined {len(CURATED_OBJECT_CLASS_CANDIDATES)} initial curated object class candidates.")

# --- Function to Load Pickle File (from previous cell, ensure it's defined or redefine) ---
def load_pickle_data(file_path):
    """
    Loads data from a specified pickle file.

    Args:
        file_path (str): The full path to the pickle file.

    Returns:
        dict: The data loaded from the pickle file, or None if an error occurs.
    """
    logging.info(f"Attempting to load pickle file from: {file_path}")
    if not os.path.exists(file_path):
        logging.error(f"Pickle file not found at {file_path}.")
        return None
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        logging.info("Successfully loaded pickle file.")
        return data
    except Exception as e:
        logging.error(f"Error loading pickle file {file_path}: {e}")
        return None

# --- Load ADE20K Index Data ---
# The INDEX_FILE_PATH variable is expected to be defined in Cell 1.
index_data = load_pickle_data(INDEX_FILE_PATH)

# Initialize global variables for class lists and mappings
TARGET_OBJECT_CLASSES = []
CLASS_NAME_TO_YOLO_ID = {}
YOLO_ID_TO_CLASS_NAME = {}
ADE20K_CLASS_INDICES_FOR_TARGETS = {} # Stores original ADE20K index for our target classes

if index_data and 'objectnames' in index_data:
    # Convert dataset object names to a set for efficient lookup.
    # ADE20K names can be compound (e.g., "foo, bar, baz"). A direct match is attempted here.
    # For more complex matching, one might need to split dataset names by comma and check parts.
    dataset_object_names_set = set(index_data['objectnames'])
    dataset_object_names_list = list(index_data['objectnames']) # To get original indices

    logging.info("Performing sanity check for curated class names against dataset object names...")
    verified_target_classes = []
    missing_classes = []

    for class_candidate in CURATED_OBJECT_CLASS_CANDIDATES:
        # Attempt a direct match first.
        # ADE20K names can be specific, e.g., "television, tv set".
        # This check is case-sensitive.
        if class_candidate in dataset_object_names_set:
            verified_target_classes.append(class_candidate)
            try:
                # Store the original ADE20K index of this class name
                ade20k_original_idx = dataset_object_names_list.index(class_candidate)
                ADE20K_CLASS_INDICES_FOR_TARGETS[class_candidate] = ade20k_original_idx
            except ValueError:
                # Should not happen if found in set, but as a safeguard
                logging.warning(f"Class '{class_candidate}' found in set but not in list (unexpected).")
        else:
            # If direct match fails, try a more lenient check (e.g., if candidate is a substring)
            # This is a simple example; more sophisticated matching might be needed if many classes are missed.
            found_variant = False
            for dataset_name in dataset_object_names_list:
                # Check if candidate is a primary part of a compound ADE20K name
                # e.g., if candidate is "television" and dataset_name is "television, tv set"
                if class_candidate == dataset_name.split(',')[0].strip():
                    verified_target_classes.append(dataset_name) # Use the full dataset name
                    try:
                        ade20k_original_idx = dataset_object_names_list.index(dataset_name)
                        ADE20K_CLASS_INDICES_FOR_TARGETS[dataset_name] = ade20k_original_idx
                        logging.info(f"Matched candidate '{class_candidate}' to dataset name '{dataset_name}'.")
                        found_variant = True
                        break
                    except ValueError:
                         logging.warning(f"Variant '{dataset_name}' for '{class_candidate}' found but not in list (unexpected).")
            if not found_variant:
                missing_classes.append(class_candidate)

    TARGET_OBJECT_CLASSES = sorted(list(set(verified_target_classes))) # Ensure uniqueness and sort

    if missing_classes:
        logging.warning(f"The following {len(missing_classes)} curated class candidates were NOT directly found or matched as primary part in dataset's objectnames and will be excluded:")
        for mc in missing_classes:
            logging.warning(f" - {mc}")
    
    logging.info(f"Total of {len(TARGET_OBJECT_CLASSES)} curated classes verified and selected for the model:")
    for i, class_name in enumerate(TARGET_OBJECT_CLASSES):
        logging.info(f"{i+1}. {class_name} (Original ADE20K Index: {ADE20K_CLASS_INDICES_FOR_TARGETS.get(class_name, 'N/A')})")

    # --- Create Class Mappings for YOLO ---
    # These mappings convert the *verified* class names to integer IDs (0 to N-1) for YOLO training.
    CLASS_NAME_TO_YOLO_ID = {name: i for i, name in enumerate(TARGET_OBJECT_CLASSES)}
    YOLO_ID_TO_CLASS_NAME = {i: name for i, name in enumerate(TARGET_OBJECT_CLASSES)}

    logging.info(f"Created YOLO ID mappings for {len(TARGET_OBJECT_CLASSES)} classes.")
    # logging.debug(f"CLASS_NAME_TO_YOLO_ID: {CLASS_NAME_TO_YOLO_ID}") # For detailed verification

    # Store objectPresence for later use if available and if indices align
    if 'objectPresence' in index_data:
        # This assumes objectPresence rows map to original ADE20K class indices
        logging.info("Stored 'objectPresence' array for potential use in data filtering.")
    else:
        logging.warning("'objectPresence' not found in index_data.")

else:
    logging.error("Failed to load index_data or 'objectnames' key is missing. Cannot proceed with class selection.")
    # Define placeholders to prevent errors in subsequent cells if index_data is None or malformed.
    index_data = {'filename': [], 'folder': [], 'scene': [], 'objectnames': [], 'objectcounts': np.array([])}
    # No classes means these will be empty, and subsequent cells should handle this.

logging.info("Cell 2 execution completed: Metadata loaded, curated classes defined, verified, and mappings created.")

2025-05-13 01:28:59,441 - INFO - Defined 85 initial curated object class candidates.
2025-05-13 01:28:59,442 - INFO - Attempting to load pickle file from: ./user-default-efs/ADE20K_2021_17_01/index_ade20k.pkl
2025-05-13 01:28:59,616 - INFO - Successfully loaded pickle file.
2025-05-13 01:28:59,618 - INFO - Performing sanity check for curated class names against dataset object names...
2025-05-13 01:28:59,618 - INFO - Matched candidate 'backpack' to dataset name 'backpack, back pack, knapsack, packsack, rucksack, haversack'.
2025-05-13 01:28:59,620 - INFO - Matched candidate 'bicycle' to dataset name 'bicycle, bike, wheel, cycle'.
2025-05-13 01:28:59,621 - INFO - Matched candidate 'bus' to dataset name 'bus, autobus, coach, charabanc, double-decker, jitney, motorbus, motorcoach, omnibus, passenger vehicle'.
2025-05-13 01:28:59,623 - INFO - Matched candidate 'truck' to dataset name 'truck, motortruck'.
2025-05-13 01:28:59,623 - INFO - Matched candidate 'airplane' to dataset name 'airplan

In [3]:
# Cell 3: Data Filtering and Splitting

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
from collections import Counter

# This cell relies on variables defined in previous cells:
# index_data, TARGET_OBJECT_CLASSES, ADE20K_CLASS_INDICES_FOR_TARGETS (from Cell 2)
# ADE20K_RAW_DATA_DIR, TRAIN_RATIO, VALIDATION_RATIO, TEST_RATIO, RANDOM_STATE_SEED (from Cell 1)

logging.info("Starting Cell 3: Data Filtering and Splitting.")

# Initialize placeholders for dataframes in case of early exit.
train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
filtered_image_data = [] # This will hold the data for images that pass the filter.

# Ensure essential data structures from previous cells are available and valid.
if not ('index_data' in globals() and index_data and
        'filename' in index_data and 'folder' in index_data and 'scene' in index_data and
        'TARGET_OBJECT_CLASSES' in globals() and TARGET_OBJECT_CLASSES):
    logging.error("Essential data from previous cells (index_data, TARGET_OBJECT_CLASSES) "
                  "is missing or invalid. Cannot proceed with data filtering and splitting.")
else:
    all_image_filenames = index_data['filename']
    all_folder_paths_from_pickle = index_data['folder'] # Paths as stored in the metadata file.
    all_scene_labels = index_data['scene']

    initial_manifest = []
    logging.info(f"Constructing initial manifest for {len(all_image_filenames)} images based on metadata.")

    # Normalize ADE20K_RAW_DATA_DIR and extract its basename.
    # This helps in intelligently joining paths, especially if metadata paths might contain
    # overlapping segments with the base directory path.
    normalized_ade20k_raw_dir = os.path.normpath(ADE20K_RAW_DATA_DIR)
    dataset_directory_basename = os.path.basename(normalized_ade20k_raw_dir)

    for i in range(len(all_image_filenames)):
        img_filename = all_image_filenames[i]
        folder_path_original = all_folder_paths_from_pickle[i]

        # Determine the correct relative path component from the metadata.
        # If the path from metadata starts with the dataset's root folder name,
        # that part is stripped to avoid duplication when joining with ADE20K_RAW_DATA_DIR.
        # This ensures that 'relative_path_to_join' is truly relative to ADE20K_RAW_DATA_DIR.
        relative_path_to_join = folder_path_original
        if folder_path_original.startswith(dataset_directory_basename + os.path.sep):
            relative_path_to_join = folder_path_original[len(dataset_directory_basename) + len(os.path.sep):]
        
        # Construct the full absolute paths for the image and its corresponding JSON annotation file.
        img_path = os.path.join(ADE20K_RAW_DATA_DIR, relative_path_to_join, img_filename)
        
        base_filename = os.path.splitext(img_filename)[0]
        json_filename = base_filename + ".json"
        json_path = os.path.join(ADE20K_RAW_DATA_DIR, relative_path_to_join, json_filename)

        initial_manifest.append({
            'image_path': img_path,
            'json_path': json_path,
            'scene': all_scene_labels[i],
            'image_idx_original': i # Preserves original index for 'objectPresence' lookup.
        })

    logging.info(f"Successfully constructed initial manifest with {len(initial_manifest)} image entries.")

    # --- Filter Manifest for Target Classes using 'objectPresence' ---
    if 'objectPresence' in index_data and hasattr(index_data['objectPresence'], 'shape'):
        object_presence_matrix = index_data['objectPresence']
        
        # Collect original ADE20K indices for the successfully verified target classes.
        target_class_original_indices = []
        if 'ADE20K_CLASS_INDICES_FOR_TARGETS' in globals():
             target_class_original_indices = [ADE20K_CLASS_INDICES_FOR_TARGETS[name] 
                                             for name in TARGET_OBJECT_CLASSES 
                                             if name in ADE20K_CLASS_INDICES_FOR_TARGETS]

        if not target_class_original_indices:
            logging.warning("No original ADE20K indices found for target classes. "
                            "Filtering based on 'objectPresence' will likely result in an empty dataset.")
        else:
            logging.info(f"Filtering images based on presence of {len(target_class_original_indices)} target classes "
                         f"using the 'objectPresence' matrix (Shape: {object_presence_matrix.shape}).")
            
            for item_data in initial_manifest:
                original_img_idx = item_data['image_idx_original']
                try:
                    # Check if any of the target classes are marked as present for this image.
                    if np.any(object_presence_matrix[target_class_original_indices, original_img_idx]):
                        filtered_image_data.append(item_data)
                except IndexError as e:
                    logging.error(f"IndexError during 'objectPresence' lookup for image index {original_img_idx}. Error: {e}. "
                                  f"Max original target index: {max(target_class_original_indices) if target_class_original_indices else 'N/A'}. "
                                  "This image will be excluded by the filter.")
            logging.info(f"Filtered down to {len(filtered_image_data)} images containing at least one target class.")
    else:
        logging.warning("'objectPresence' array not found or is invalid in 'index_data'. "
                        "Efficient filtering for target classes cannot be performed. "
                        "Consider implementing JSON-based filtering as an alternative if this is problematic. "
                        "For now, proceeding with data that passed any prior steps (if any).")
        # If filtering is critical and objectPresence is missing, this might mean filtered_image_data remains empty or unchanged.

    if not filtered_image_data:
        logging.error("No images found after filtering step or initial manifest was empty. "
                      "Subsequent data splitting and processing will be skipped.")
    else:
        manifest_df = pd.DataFrame(filtered_image_data)

        # --- Split Data into Training, Validation, and Testing Sets ---
        logging.info(f"Attempting to split {len(manifest_df)} filtered images into train/validation/test sets.")
        
        # Calculate ratio for the combined validation and test set.
        remaining_ratio = VALIDATION_RATIO + TEST_RATIO
        if np.isclose(remaining_ratio, 0.0) or remaining_ratio > 1.0: # Basic check
             logging.error(f"Invalid validation/test ratios ({VALIDATION_RATIO}, {TEST_RATIO}). Sum is {remaining_ratio}. Cannot split.")
        else:
            stratify_by = None
            if 'scene' in manifest_df.columns:
                scene_counts = Counter(manifest_df['scene'])
                num_unique_scenes = len(scene_counts)
                # Heuristic for stratification: requires enough samples in most scene categories.
                # sklearn's train_test_split needs at least 2 members for the smallest group in stratification.
                scenes_suitable_for_stratify = [s for s, c in scene_counts.items() if c >= max(2, int(1/(VALIDATION_RATIO+TEST_RATIO)))] # Ensure enough for val+test portion
                
                if len(scenes_suitable_for_stratify) > num_unique_scenes * 0.5 and len(scenes_suitable_for_stratify) > 1:
                    stratify_by = manifest_df['scene']
                    logging.info(f"Attempting stratified split by 'scene'. Unique scenes: {num_unique_scenes}. Scenes suitable for stratification: {len(scenes_suitable_for_stratify)}.")
                else:
                    logging.warning(f"Stratification by 'scene' may not be effective (Unique: {num_unique_scenes}, Suitable: {len(scenes_suitable_for_stratify)}). Using random split for train/temp_val_test.")

            try:
                # First split: separate training set from a temporary set (validation + test).
                train_df, temp_val_test_df = train_test_split(
                    manifest_df,
                    test_size=remaining_ratio,
                    random_state=RANDOM_STATE_SEED,
                    stratify=stratify_by
                )
            except ValueError as e: # Handles cases where stratification fails (e.g., too few samples in a class).
                logging.warning(f"Stratification for train/temp_val_test split failed: {e}. Retrying with random split.")
                train_df, temp_val_test_df = train_test_split(
                    manifest_df,
                    test_size=remaining_ratio,
                    random_state=RANDOM_STATE_SEED,
                    stratify=None # Fallback to random split.
                )
            
            # Second split: separate validation and test sets from the temporary set.
            if not temp_val_test_df.empty and not np.isclose(remaining_ratio, 0.0):
                # Adjust test_size for the second split to be relative to temp_val_test_df.
                relative_test_size = TEST_RATIO / remaining_ratio
                
                val_test_stratify_by = None
                if stratify_by is not None : # Attempt to stratify second split if first was stratified
                    temp_scene_counts = Counter(temp_val_test_df['scene'])
                    temp_scenes_suitable = [s for s,c in temp_scene_counts.items() if c >= 2] # Min 2 for a binary split
                    if len(temp_scenes_suitable) > len(temp_scene_counts) * 0.5 and len(temp_scenes_suitable) > 1:
                        val_test_stratify_by = temp_val_test_df['scene']
                    else:
                        logging.warning("Stratification for val/test split may not be effective. Using random split for val/test.")


                try:
                    val_df, test_df = train_test_split(
                        temp_val_test_df,
                        test_size=relative_test_size,
                        random_state=RANDOM_STATE_SEED,
                        stratify=val_test_stratify_by
                    )
                except ValueError as e:
                    logging.warning(f"Stratification for val/test split failed: {e}. Retrying with random split.")
                    val_df, test_df = train_test_split(
                        temp_val_test_df,
                        test_size=relative_test_size,
                        random_state=RANDOM_STATE_SEED,
                        stratify=None # Fallback to random split.
                    )
            elif temp_val_test_df.empty:
                 logging.warning("Temporary validation/test dataframe (temp_val_test_df) is empty. Validation and Test sets will also be empty.")
                 val_df = pd.DataFrame(columns=manifest_df.columns if not manifest_df.empty else None)
                 test_df = pd.DataFrame(columns=manifest_df.columns if not manifest_df.empty else None)


            logging.info("Data splitting process completed.")
            logging.info(f"  Total images after filtering: {len(manifest_df)}")
            logging.info(f"  Training set size: {len(train_df)}")
            logging.info(f"  Validation set size: {len(val_df)}")
            logging.info(f"  Test set size: {len(test_df)}")

            if train_df.empty or val_df.empty or test_df.empty:
                logging.warning("One or more dataset splits (train, validation, test) are empty. "
                                "This may indicate an issue with the filtering, splitting ratios, or very small initial dataset.")

logging.info("Cell 3 execution completed.")

2025-05-12 11:03:14,825 - INFO - Starting Cell 3: Data Filtering and Splitting.
2025-05-12 11:03:14,829 - INFO - Constructing initial manifest for 27574 images based on metadata.
2025-05-12 11:03:14,958 - INFO - Successfully constructed initial manifest with 27574 image entries.
2025-05-12 11:03:14,959 - INFO - Filtering images based on presence of 72 target classes using the 'objectPresence' matrix (Shape: (3688, 27574)).
2025-05-12 11:03:15,354 - INFO - Filtered down to 26692 images containing at least one target class.
2025-05-12 11:03:15,376 - INFO - Attempting to split 26692 filtered images into train/validation/test sets.
2025-05-12 11:03:15,384 - INFO - Attempting stratified split by 'scene'. Unique scenes: 1642. Scenes suitable for stratification: 973.
2025-05-12 11:03:15,405 - WARNING - Stratification for train/temp_val_test split failed: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.. Retry

In [4]:
# Cell 4: YOLOv8 Dataset Conversion Functions

import json
from PIL import Image
import logging
import os # For os.path.exists

# Ensure CLASS_NAME_TO_YOLO_ID and TARGET_OBJECT_CLASSES are available from Cell 2.
# TARGET_OBJECT_CLASSES should be converted to a set for efficient lookup.
TARGET_OBJECT_CLASSES_SET = set(TARGET_OBJECT_CLASSES) if 'TARGET_OBJECT_CLASSES' in globals() and TARGET_OBJECT_CLASSES else set()

logging.info("Starting Cell 4: Defining YOLOv8 Dataset Conversion Functions.")

def read_json_annotation(json_path):
    """
    Reads and parses a JSON annotation file.

    Args:
        json_path (str): The full path to the JSON annotation file.

    Returns:
        dict: The parsed JSON data (specifically the list of object annotations),
              or None if the file does not exist or an error occurs during parsing.
    """
    if not os.path.exists(json_path):
        logging.warning(f"Annotation JSON file not found: {json_path}")
        return None
    try:
        with open(json_path, 'r') as f:
            annotation_data = json.load(f)
        # The actual object annotations are usually nested, e.g., under 'annotation' -> 'object'
        if 'annotation' in annotation_data and 'object' in annotation_data['annotation']:
            return annotation_data['annotation']['object']
        else:
            logging.warning(f"JSON file {json_path} does not have the expected structure ('annotation' -> 'object').")
            return None
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON from file {json_path}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred while reading {json_path}: {e}")
        return None

def convert_to_yolo_segmentation_format(json_path, image_path, class_name_to_yolo_id_map, target_classes_set):
    """
    Converts ADE20K object annotations from a JSON file to YOLOv8 segmentation format.

    Args:
        json_path (str): Path to the JSON annotation file.
        image_path (str): Path to the corresponding image file (for dimensions).
        class_name_to_yolo_id_map (dict): Mapping from class names to YOLO integer IDs.
        target_classes_set (set): A set of target class names to include.

    Returns:
        list: A list of strings, where each string is a YOLO formatted annotation
              (class_id norm_x1 norm_y1 norm_x2 norm_y2 ...).
              Returns an empty list if no valid objects are found or an error occurs.
    """
    yolo_annotations = []

    # 1. Load image to get dimensions for normalization
    try:
        with Image.open(image_path) as img:
            img_width, img_height = img.size
    except FileNotFoundError:
        logging.error(f"Image file not found: {image_path} while processing {json_path}.")
        return []
    except Exception as e:
        logging.error(f"Error opening image {image_path}: {e}")
        return []

    if img_width == 0 or img_height == 0:
        logging.error(f"Image {image_path} has zero width or height.")
        return []

    # 2. Read object annotations from JSON
    object_annotations = read_json_annotation(json_path)
    if not object_annotations:
        # read_json_annotation would have logged the specific error
        return []

    # 3. Process each annotated object
    for ann_object in object_annotations:
        try:
            # Use 'name' for the corrected object name, as identified from utils_ade20k.py
            object_name = ann_object.get('name', '').strip()
            
            # Check if it's a primary object (not a part)
            # In ADE20K JSON, 'parts' is a dict, and 'part_level' indicates if it's a part.
            # Level '0' means it is a main object.
            part_level_str = ann_object.get('parts', {}).get('part_level', '-1')
            is_primary_object = (int(part_level_str) == 0)

            if is_primary_object and object_name in target_classes_set:
                yolo_class_id = class_name_to_yolo_id_map.get(object_name)
                if yolo_class_id is None:
                    # This should not happen if target_classes_set is derived from class_name_to_yolo_id_map keys
                    logging.warning(f"Object name '{object_name}' in target set but not in ID map. Skipping.")
                    continue

                polygon_data = ann_object.get('polygon')
                if not polygon_data:
                    logging.debug(f"Object '{object_name}' in {json_path} has no polygon data. Skipping.")
                    continue

                polygon_x_coords = polygon_data.get('x', [])
                polygon_y_coords = polygon_data.get('y', [])

                if not polygon_x_coords or not polygon_y_coords or len(polygon_x_coords) != len(polygon_y_coords):
                    logging.debug(f"Invalid or empty polygon coordinates for '{object_name}' in {json_path}. Skipping.")
                    continue
                
                if len(polygon_x_coords) < 3: # A polygon must have at least 3 points
                    logging.debug(f"Polygon for '{object_name}' in {json_path} has fewer than 3 points. Skipping.")
                    continue

                # Normalize and format polygon coordinates
                normalized_points = []
                for px, py in zip(polygon_x_coords, polygon_y_coords):
                    norm_x = round(float(px) / img_width, 6)  # Keep reasonable precision
                    norm_y = round(float(py) / img_height, 6)
                    # Ensure coordinates are within [0, 1] bounds
                    norm_x = max(0.0, min(1.0, norm_x))
                    norm_y = max(0.0, min(1.0, norm_y))
                    normalized_points.extend([norm_x, norm_y])
                
                if normalized_points:
                    yolo_annotations.append(f"{yolo_class_id} " + " ".join(map(str, normalized_points)))
            
        except (TypeError, ValueError) as e:
            logging.warning(f"Skipping object in {json_path} due to data error: {e}. Object data: {ann_object}")
        except Exception as e:
            logging.error(f"Unexpected error processing an object in {json_path}: {e}. Object data: {ann_object}")
            
    return yolo_annotations

# Example usage (for testing purposes, actual use will be in Cell 5)
# if 'train_df' in globals() and not train_df.empty:
#     sample_item = train_df.iloc[0]
#     logging.info(f"\n--- Example Conversion Test ---")
#     logging.info(f"Testing with image: {sample_item['image_path']}")
#     logging.info(f"Annotation JSON: {sample_item['json_path']}")
#     if TARGET_OBJECT_CLASSES_SET and CLASS_NAME_TO_YOLO_ID:
#         example_yolo_data = convert_to_yolo_segmentation_format(
#             sample_item['json_path'],
#             sample_item['image_path'],
#             CLASS_NAME_TO_YOLO_ID,
#             TARGET_OBJECT_CLASSES_SET
#         )
#         if example_yolo_data:
#             logging.info(f"Example YOLO formatted annotations ({len(example_yolo_data)} objects found):")
#             for line in example_yolo_data[:3]: # Print first 3 annotations
#                 logging.info(line)
#         else:
#             logging.info("No target objects found or error in example conversion.")
#     else:
#         logging.warning("TARGET_OBJECT_CLASSES_SET or CLASS_NAME_TO_YOLO_ID not defined for example.")
#     logging.info(f"--- End Example Conversion Test ---\n")


logging.info("Cell 4 execution completed: YOLOv8 conversion functions are now defined.")

2025-05-12 11:03:15,434 - INFO - Starting Cell 4: Defining YOLOv8 Dataset Conversion Functions.
2025-05-12 11:03:15,436 - INFO - Cell 4 execution completed: YOLOv8 conversion functions are now defined.


In [5]:
# Cell 5: Generate YOLOv8 Labels and Dataset Structure

import os
import shutil # For copying files
import logging
import pandas as pd # Expected if train_df, val_df, test_df are pandas DataFrames

# Ensure variables from previous cells are available:
# - train_df, val_df, test_df (DataFrames from Cell 3)
# - CLASS_NAME_TO_YOLO_ID, TARGET_OBJECT_CLASSES_SET (from Cell 2)
# - YOLO_IMAGES_DIR, YOLO_LABELS_DIR (from Cell 1)
# - convert_to_yolo_segmentation_format (function from Cell 4)

logging.info("Starting Cell 5: Generating YOLOv8 Labels and Copying Images.")

def process_dataset_split(dataframe, subset_name, class_to_id_map, target_classes_set,
                          yolo_images_root, yolo_labels_root):
    """
    Processes a specific dataset split (train, val, or test).
    Generates YOLO label files and copies images to the target YOLO directory structure.

    Args:
        dataframe (pd.DataFrame): DataFrame containing 'image_path' and 'json_path' for the split.
        subset_name (str): The name of the subset (e.g., 'train', 'val', 'test').
        class_to_id_map (dict): Mapping from class names to YOLO integer IDs.
        target_classes_set (set): A set of target class names.
        yolo_images_root (str): Root directory for YOLO images.
        yolo_labels_root (str): Root directory for YOLO labels.
    """
    if not isinstance(dataframe, pd.DataFrame) or dataframe.empty:
        logging.warning(f"DataFrame for '{subset_name}' is empty or not a DataFrame. Skipping processing for this subset.")
        return

    target_img_dir = os.path.join(yolo_images_root, subset_name)
    target_lbl_dir = os.path.join(yolo_labels_root, subset_name)

    # Ensure target directories exist (should have been created in Cell 1)
    os.makedirs(target_img_dir, exist_ok=True)
    os.makedirs(target_lbl_dir, exist_ok=True)

    logging.info(f"Processing '{subset_name}' set: {len(dataframe)} images.")
    processed_count = 0
    error_count = 0

    for index, row in dataframe.iterrows():
        try:
            image_path_original = row['image_path']
            json_path_original = row['json_path']

            if not os.path.exists(image_path_original):
                logging.warning(f"Original image file not found: {image_path_original}. Skipping.")
                error_count += 1
                continue
            # json_path existence is checked within convert_to_yolo_segmentation_format

            # 1. Convert annotations to YOLO format
            yolo_annotation_strings = convert_to_yolo_segmentation_format(
                json_path_original,
                image_path_original,
                class_to_id_map,
                target_classes_set
            )

            # 2. Determine output paths
            base_image_filename = os.path.basename(image_path_original)
            label_filename = os.path.splitext(base_image_filename)[0] + ".txt"
            
            output_label_path = os.path.join(target_lbl_dir, label_filename)
            output_image_path = os.path.join(target_img_dir, base_image_filename)

            # 3. Write label file (even if empty, as YOLO expects it)
            with open(output_label_path, 'w') as f:
                for line in yolo_annotation_strings:
                    f.write(line + "\n")

            # 4. Copy image file
            shutil.copy(image_path_original, output_image_path)
            
            processed_count += 1
            if processed_count % 500 == 0: # Log progress every 500 images
                logging.info(f"  Processed {processed_count}/{len(dataframe)} images for '{subset_name}' set...")

        except Exception as e:
            logging.error(f"Failed to process item {index} (image: {row.get('image_path', 'N/A')}) in '{subset_name}' set: {e}")
            error_count += 1
            # Continue with the next file

    logging.info(f"Finished processing '{subset_name}' set. Successfully processed: {processed_count}. Errors: {error_count}.")


# Check if necessary DataFrames are defined and not empty
data_splits_to_process = {}
if 'train_df' in globals() and isinstance(train_df, pd.DataFrame) and not train_df.empty:
    data_splits_to_process['train'] = train_df
else:
    logging.warning("train_df is not defined or is empty. Training set will not be processed.")

if 'val_df' in globals() and isinstance(val_df, pd.DataFrame) and not val_df.empty:
    data_splits_to_process['val'] = val_df
else:
    logging.warning("val_df is not defined or is empty. Validation set will not be processed.")

if 'test_df' in globals() and isinstance(test_df, pd.DataFrame) and not test_df.empty:
    data_splits_to_process['test'] = test_df
else:
    logging.warning("test_df is not defined or is empty. Test set will not be processed.")

# Check for critical supporting variables from Cell 2
if 'CLASS_NAME_TO_YOLO_ID' not in globals() or not CLASS_NAME_TO_YOLO_ID:
    logging.error("CLASS_NAME_TO_YOLO_ID is not defined or empty. Cannot proceed with label generation.")
elif 'TARGET_OBJECT_CLASSES_SET' not in globals() or not TARGET_OBJECT_CLASSES_SET:
    logging.error("TARGET_OBJECT_CLASSES_SET is not defined or empty. Cannot proceed with label generation.")
else:
    # Process each dataset split
    for subset_name, dataframe_to_process in data_splits_to_process.items():
        process_dataset_split(
            dataframe_to_process,
            subset_name,
            CLASS_NAME_TO_YOLO_ID,
            TARGET_OBJECT_CLASSES_SET,
            YOLO_IMAGES_DIR, # Defined in Cell 1
            YOLO_LABELS_DIR  # Defined in Cell 1
        )

logging.info("Cell 5 execution completed: YOLOv8 dataset generation attempted for all splits.")

2025-05-12 11:03:15,451 - INFO - Starting Cell 5: Generating YOLOv8 Labels and Copying Images.
2025-05-12 11:03:15,453 - INFO - Processing 'train' set: 21353 images.
2025-05-12 11:03:26,010 - INFO -   Processed 500/21353 images for 'train' set...
2025-05-12 11:03:36,109 - INFO -   Processed 1000/21353 images for 'train' set...
2025-05-12 11:03:46,299 - INFO -   Processed 1500/21353 images for 'train' set...
2025-05-12 11:03:55,759 - ERROR - An unexpected error occurred while reading ./user-default-efs/ADE20K_2021_17_01/images/ADE/training/urban/apartment_building__outdoor/ADE_train_00024795.json: 'utf-8' codec can't decode byte 0xf1 in position 83348: invalid continuation byte
2025-05-12 11:03:56,716 - INFO -   Processed 2000/21353 images for 'train' set...
2025-05-12 11:04:06,830 - INFO -   Processed 2500/21353 images for 'train' set...
2025-05-12 11:04:17,038 - INFO -   Processed 3000/21353 images for 'train' set...
2025-05-12 11:04:27,400 - INFO -   Processed 3500/21353 images for '

In [4]:
# Cell 6: Create YOLOv8 Dataset YAML File

import os
import logging
import yaml # PyYAML library for YAML manipulation

# Ensure variables from previous cells are available:
# - YOLO_DATASET_ROOT_DIR (from Cell 1)
# - YOLO_ID_TO_CLASS_NAME (from Cell 2, mapping integer ID to class name string)
# - TARGET_OBJECT_CLASSES (from Cell 2, list of class name strings in order)

logging.info("Starting Cell 6: Creating YOLOv8 Dataset YAML File.")

# Define the name and path for the YAML file.
# It's common to place this in the root of the YOLO dataset directory.
yaml_file_name = "ade20k_yolo.yaml"
yaml_file_path = os.path.join(YOLO_DATASET_ROOT_DIR, yaml_file_name)

# Check if critical variables are defined
if 'YOLO_DATASET_ROOT_DIR' not in globals() or not YOLO_DATASET_ROOT_DIR:
    logging.error("YOLO_DATASET_ROOT_DIR is not defined. Cannot create YAML file.")
elif ('YOLO_ID_TO_CLASS_NAME' not in globals() or not YOLO_ID_TO_CLASS_NAME or
      'TARGET_OBJECT_CLASSES' not in globals() or not TARGET_OBJECT_CLASSES):
    logging.error("Class name mappings (YOLO_ID_TO_CLASS_NAME or TARGET_OBJECT_CLASSES) "
                  "are not defined or empty. Cannot create YAML file.")
else:
    try:
        # Ensure YOLO_DATASET_ROOT_DIR is an absolute path for robustness in the YAML file.
        abs_yolo_dataset_root_dir = os.path.abspath(YOLO_DATASET_ROOT_DIR)

        # Define paths relative to the YOLO_DATASET_ROOT_DIR for train/val/test sets.
        # These are standard subdirectories created in Cell 1 and populated in Cell 5.
        train_images_rel_path = os.path.join('images', 'train')
        val_images_rel_path = os.path.join('images', 'val')
        test_images_rel_path = os.path.join('images', 'test')

        # Prepare the list of class names in the correct order (index = class_id).
        # TARGET_OBJECT_CLASSES should already be sorted if it was derived from a sorted list
        # or if CLASS_NAME_TO_YOLO_ID was built from it sequentially.
        # YOLO_ID_TO_CLASS_NAME provides the mapping {0: name0, 1: name1, ...}
        # The number of classes is the length of TARGET_OBJECT_CLASSES.
        num_classes = len(TARGET_OBJECT_CLASSES)
        class_names_list = [None] * num_classes
        for i in range(num_classes):
            class_name = YOLO_ID_TO_CLASS_NAME.get(i)
            if class_name is None:
                logging.error(f"Error: Class ID {i} not found in YOLO_ID_TO_CLASS_NAME map. Inconsistent state.")
                raise ValueError(f"Missing class name for ID {i} in YOLO_ID_TO_CLASS_NAME.")
            class_names_list[i] = class_name
        
        # Construct the data dictionary for the YAML file.
        yaml_data = {
            'path': abs_yolo_dataset_root_dir,  # Absolute path to the dataset root
            'train': train_images_rel_path,     # Path to train images relative to 'path'
            'val': val_images_rel_path,         # Path to val images relative to 'path'
            'test': test_images_rel_path,       # Path to test images relative to 'path'
            'nc': num_classes,                  # Number of classes
            'names': class_names_list           # List of class names
        }

        # Write the dictionary to a YAML file.
        # Using 'sort_keys=False' to maintain the order as defined in yaml_data.
        # 'default_flow_style=None' makes lists appear on new lines typically.
        with open(yaml_file_path, 'w') as f:
            yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=None)

        logging.info(f"Successfully created YOLOv8 dataset YAML file at: {yaml_file_path}")
        logging.info("YAML file content:")
        # Reading and printing the file content for verification.
        with open(yaml_file_path, 'r') as f_read:
            logging.info(f"\n{f_read.read()}")

    except ImportError:
        logging.error("The 'PyYAML' library is not installed. Cannot write YAML file using yaml.dump().")
        logging.info("Attempting to write YAML content as a formatted string as a fallback.")
        # Fallback: Manually create YAML string content if PyYAML is not available.
        # This is less robust than using the PyYAML library but avoids an external dependency if not present.
        yaml_string_content = f"path: {abs_yolo_dataset_root_dir}\n"
        yaml_string_content += f"train: {train_images_rel_path}\n"
        yaml_string_content += f"val: {val_images_rel_path}\n"
        yaml_string_content += f"test: {test_images_rel_path}\n\n"
        yaml_string_content += f"nc: {num_classes}\n"
        yaml_string_content += "names:\n"
        for i, name in enumerate(class_names_list):
            yaml_string_content += f"  - \"{name}\" # ID {i}\n" # Enclosing names in quotes if they contain special chars.
        
        try:
            with open(yaml_file_path, 'w') as f_str:
                f_str.write(yaml_string_content)
            logging.info(f"Successfully wrote YAML content as string to: {yaml_file_path}")
            logging.info("Fallback YAML file content:")
            logging.info(f"\n{yaml_string_content}")
        except Exception as e_str:
            logging.error(f"Failed to write YAML content as string: {e_str}")

    except ValueError as ve: # Catch potential ValueError from class name list construction
        logging.error(f"A ValueError occurred during YAML data preparation: {ve}")
    except Exception as e:
        logging.error(f"An unexpected error occurred while creating the YAML file: {e}")

logging.info("Cell 6 execution completed.")

2025-05-13 01:29:17,780 - INFO - Starting Cell 6: Creating YOLOv8 Dataset YAML File.
2025-05-13 01:29:17,783 - INFO - Successfully created YOLOv8 dataset YAML file at: ./ADE20K_YOLOv8_Dataset/ade20k_yolo.yaml
2025-05-13 01:29:17,784 - INFO - YAML file content:
2025-05-13 01:29:17,785 - INFO - 
path: /home/sagemaker-user/ADE20K_YOLOv8_Dataset
train: images/train
val: images/val
test: images/test
nc: 72
names: ['airplane, aeroplane, plane', 'backpack, back pack, knapsack, packsack, rucksack,
    haversack', bed, bench, 'bicycle, bike, wheel, cycle', bird, blender, boat, book,
  bottle, bowl, building, 'bus, autobus, coach, charabanc, double-decker, jitney,
    motorbus, motorcoach, omnibus, passenger vehicle', cabinet, car, cat, ceiling,
  chair, clock, 'cow, moo-cow', cup, cushion, desk, 'dog, domestic dog, Canis familiaris',
  door, fence, floor, fork, handbag, 'horse, Equus caballus', house, keyboard, knife,
  'laptop, laptop computer', light, light source, microwave, monitor, mouse, 

In [ ]:
# Cell 7: Setup Ultralytics YOLOv8

import logging
import subprocess
import sys

logging.info("Starting Cell 7: Setting up Ultralytics YOLOv8.")

!pip install ultralytics
!sudo apt-get update && sudo apt-get install -y libgl1-mesa-glx

# --- Installation Check and Attempt (Optional) ---
# The Ultralytics package is required. If it's not already installed in the
# environment, the following line can be uncommented and run.
# In managed environments like SageMaker, it might be pre-installed or
# require a specific installation procedure.

# try:
#     logging.info("Checking if ultralytics is installed...")
#     import ultralytics
#     logging.info(f"Ultralytics YOLOv8 already installed. Version: {ultralytics.__version__}")
# except ImportError:
#     logging.warning("Ultralytics YOLOv8 not found. Attempting to install...")
#     try:
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
#         logging.info("Successfully installed ultralytics.")
#         import ultralytics # Verify import after installation
#         logging.info(f"Ultralytics YOLOv8 successfully imported after installation. Version: {ultralytics.__version__}")
#     except Exception as e:
#         logging.error(f"Failed to install or import ultralytics: {e}")
#         logging.error("Please ensure 'ultralytics' is installed in the environment to proceed with model training.")
#         # Raising an error here might be too disruptive, logging is preferred for now.
#         # The import attempt below will ultimately determine if it's usable.

# --- Import YOLO Class ---
try:
    from ultralytics import YOLO
    logging.info("Successfully imported YOLO from ultralytics.")
    # Displaying YOLOv8 environment details (optional)
    # from ultralytics.utils.checks import check_requirements, check_version, check_pytorch # check_yolo
    # check_requirements('ultralytics') # This can be verbose
    # check_yolo() # This prints a lot of info, might be too much for standard log.
except ImportError as e:
    logging.error(f"Failed to import YOLO from ultralytics: {e}")
    logging.error("This usually means 'ultralytics' is not installed correctly or there's an environment issue.")
    logging.error("Please install it using 'pip install ultralytics' and ensure the Python environment is configured correctly.")
    # It might be prudent to raise an exception here if YOLO cannot be imported,
    # as subsequent cells will fail. For now, logging the error.
    # raise e # Uncomment to make this cell fail if import fails.
except Exception as e:
    logging.error(f"An unexpected error occurred during YOLO import: {e}")
    # raise e

logging.info("Cell 7 execution completed: Ultralytics YOLOv8 setup attempted.")

2025-05-13 01:29:21,494 - INFO - Starting Cell 7: Setting up Ultralytics YOLOv8.


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease    
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease              
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


2025-05-13 01:29:29,039 - INFO - Successfully imported YOLO from ultralytics.
2025-05-13 01:29:29,040 - INFO - Cell 7 execution completed: Ultralytics YOLOv8 setup attempted.


In [ ]:
# Cell 8: Train YOLOv8s-seg Model

import os
import logging
import torch # To check CUDA availability

# Ensure YOLO class is imported (from Cell 7) and yaml_file_path is defined (from Cell 6)

logging.info("Starting Cell 8: Training YOLOv8s-seg Model.")

# --- Training Configuration ---
# These parameters can be adjusted based on requirements and available resources.
TRAINING_EPOCHS = 150  # Number of epochs to train for. Start with a moderate number.
IMAGE_SIZE = 640      # Input image size for training.
BATCH_SIZE = 16       # Batch size. Adjust based on GPU memory. (e.g., 8, 16, 32)
# Check if CUDA is available and set the device accordingly.
DEVICE = '0' if torch.cuda.is_available() else 'cpu' # Use GPU '0' if available, else CPU.
if DEVICE == 'cpu':
    logging.warning("CUDA GPU not detected by PyTorch. Training will run on CPU, which will be very slow.")
    logging.warning("If a GPU is present, please ensure PyTorch is correctly installed with CUDA support and the GPU drivers are up to date.")
else:
    logging.info(f"CUDA GPU detected. Training will run on device: {DEVICE}")

# Define project and run names for organizing training outputs.
# YOLOv8 saves runs under '{PROJECT_DIR}/{RUN_NAME}'
PROJECT_DIR = os.path.join('.', 'YOLOv8_Training_Runs_ADE20K') # Root for all training runs for this project
RUN_NAME = 'ade20k_yolov8s_seg_run12' # Specific name for this particular training run

# Path to the dataset configuration YAML file (created in Cell 6).
# Ensure 'yaml_file_path' is correctly carried over from Cell 6.
if 'yaml_file_path' not in globals() or not os.path.exists(yaml_file_path):
    logging.error(f"Dataset YAML configuration file not found at expected path: {globals().get('yaml_file_path', 'Path not defined')}.")
    logging.error("Please ensure Cell 6 was executed successfully and yaml_file_path is correct.")
    # Halt further execution in this cell if YAML is missing.
    # This can be done by raising an error or simply not proceeding.
    # For now, we'll log and let it potentially fail at model.train() if path is None.
    dataset_yaml_path = None
else:
    dataset_yaml_path = yaml_file_path
    logging.info(f"Using dataset YAML configuration from: {dataset_yaml_path}")

# --- Initialize and Train the Model ---
if dataset_yaml_path:
    try:
        # Load a pre-trained YOLOv8s segmentation model.
        # Using '.pt' loads pre-trained weights, which is recommended for custom datasets.
        model = YOLO('yolov8s-seg.pt')
        logging.info("Initialized YOLOv8s-seg model with pre-trained weights (yolov8s-seg.pt).")

        logging.info(f"Starting model training with the following parameters:")
        logging.info(f"  Data YAML: {dataset_yaml_path}")
        logging.info(f"  Epochs: {TRAINING_EPOCHS}")
        logging.info(f"  Image Size: {IMAGE_SIZE}")
        logging.info(f"  Batch Size: {BATCH_SIZE}")
        logging.info(f"  Device: {DEVICE}")
        logging.info(f"  Project Directory: {PROJECT_DIR}")
        logging.info(f"  Run Name: {RUN_NAME}")
        
        # Start training.
        # The training process will output logs to the console and save results
        # (weights, metrics, plots) in 'PROJECT_DIR/RUN_NAME'.
        # Other arguments like 'workers', 'patience' (for early stopping) can be added.
        model.train(
            data=dataset_yaml_path,
            epochs=TRAINING_EPOCHS,
            imgsz=IMAGE_SIZE,
            batch=BATCH_SIZE,
            device=DEVICE,
            project=PROJECT_DIR,
            name=RUN_NAME,
            patience=20, # Number of epochs to wait for improvement before early stopping (e.g., 20-30)
            exist_ok=False # If True, allows overwriting existing run with same name. Set to False to prevent accidental overwrites.
                           # If False and run exists, it will create e.g., run_name2, run_name3 ...
        )

        logging.info(f"Model training initiated. Check console output for progress.")
        logging.info(f"Training results, including weights and metrics, will be saved in: "
                     f"{os.path.abspath(os.path.join(PROJECT_DIR, RUN_NAME))}")

    except NameError as ne: # Catch if YOLO class was not imported
        logging.error(f"NameError: {ne}. This might indicate that the YOLO class was not successfully imported in Cell 7.")
    except FileNotFoundError as fnfe: # Catch if model weights or data YAML not found
        logging.error(f"FileNotFoundError: {fnfe}. Ensure 'yolov8s-seg.pt' is accessible and data YAML path is correct.")
    except Exception as e:
        logging.error(f"An unexpected error occurred during model initialization or training: {e}")
else:
    logging.error("Cannot start training as dataset YAML path is not available.")

logging.info("Cell 8 execution completed: Model training process has been started (if all configurations were correct).")

2025-05-12 11:12:28,935 - INFO - Starting Cell 8: Training YOLOv8s-seg Model.
2025-05-12 11:12:29,118 - INFO - CUDA GPU detected. Training will run on device: 0
2025-05-12 11:12:29,119 - INFO - Using dataset YAML configuration from: ./ADE20K_YOLOv8_Dataset/ade20k_yolo.yaml
2025-05-12 11:12:29,467 - INFO - Initialized YOLOv8s-seg model with pre-trained weights (yolov8s-seg.pt).
2025-05-12 11:12:29,468 - INFO - Starting model training with the following parameters:
2025-05-12 11:12:29,469 - INFO -   Data YAML: ./ADE20K_YOLOv8_Dataset/ade20k_yolo.yaml
2025-05-12 11:12:29,469 - INFO -   Epochs: 150
2025-05-12 11:12:29,470 - INFO -   Image Size: 640
2025-05-12 11:12:29,470 - INFO -   Batch Size: 16
2025-05-12 11:12:29,471 - INFO -   Device: 0
2025-05-12 11:12:29,472 - INFO -   Project Directory: ./YOLOv8_Training_Runs_ADE20K
2025-05-12 11:12:29,473 - INFO -   Run Name: ade20k_yolov8s_seg_run1


Ultralytics 8.3.131 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ADE20K_YOLOv8_Dataset/ade20k_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ade20k_yolov8s_seg_run12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=Tr

train: Scanning /home/sagemaker-user/ADE20K_YOLOv8_Dataset/labels/train.cache... 21353 images, 142 backgrounds, 0 corrupt: 100%|██████████| 21353/21353 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 750.3±281.3 MB/s, size: 173.1 KB)


val: Scanning /home/sagemaker-user/ADE20K_YOLOv8_Dataset/labels/val.cache... 2669 images, 15 backgrounds, 0 corrupt: 100%|██████████| 2669/2669 [00:00<?, ?it/s]


...

      2/150      5.68G       1.13      2.653      1.672      1.282         77        640: 100%|██████████| 1335/1335 [08:19<00:00,  2.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 84/84 [00:32<00:00,  2.58it/s]


                   all       2669      19046      0.436       0.23      0.225      0.148      0.409      0.215      0.207      0.119

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      3/150       5.9G      1.213      2.797      1.765      1.337        126        640: 100%|██████████| 1335/1335 [08:16<00:00,  2.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 84/84 [00:32<00:00,  2.61it/s]


                   all       2669      19046      0.475      0.186      0.184      0.116      0.463      0.173      0.167     0.0886

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      4/150      7.01G      1.268      2.889      1.874      1.387        209        640:  11%|█         | 147/1335 [00:54<07:17,  2.72it/s]

In [6]:
import torch # Add this if not already imported in this new session
IMAGE_SIZE = 640      # Image size used during the 88-epoch training
BATCH_SIZE = 16       # Batch size used (can be different for eval, but consistency is fine)
DEVICE = '0' if torch.cuda.is_available() else 'cpu'

# Ensure PROJECT_DIR and RUN_NAME are correctly defined here if not in Cell 1
# PROJECT_DIR = './YOLOv8_Training_Runs_ADE20K' # Already defined if you added to Cell 1
# RUN_NAME = 'ade20k_yolov8s_seg_run1'      # Already defined if you added to Cell 1

In [7]:
# Cell 9: Load Trained Model and Evaluate on Validation and Test Sets

import os
import logging
import torch # To check device

# Ensure YOLO class is imported (from Cell 7)
# Ensure PROJECT_DIR, RUN_NAME (from Cell 8), and yaml_file_path (from Cell 6) are available.
# Ensure IMAGE_SIZE, BATCH_SIZE, DEVICE (from Cell 8) are available for consistency if needed for val.

logging.info("Starting Cell 9: Loading best trained model and evaluating.")

# --- Configuration for Evaluation ---
# Path to the best trained model weights.
# These are typically saved by YOLOv8 in 'project/name/weights/best.pt'.
if 'PROJECT_DIR' not in globals() or 'RUN_NAME' not in globals():
    logging.error("PROJECT_DIR or RUN_NAME not defined from training cell. Cannot locate best model.")
    # Terminate or set best_model_path to None to prevent further execution in this cell.
    best_model_path = None
else:
    best_model_path = os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'best.pt')
    logging.info(f"Path to the best model weights: {best_model_path}")

# Check if the best model file exists.
if not (best_model_path and os.path.exists(best_model_path)):
    logging.error(f"Best model weights file not found at: {best_model_path}")
    logging.error("Please ensure training completed successfully and the path is correct.")
    # Store None or empty dicts to prevent crashes in Cell 10
    val_metrics = None
    test_metrics = None 
else:
    # --- Load the Best Trained Model ---
    try:
        model = YOLO(best_model_path) # Load the custom trained model
        logging.info(f"Successfully loaded the best trained model from: {best_model_path}")

        # --- Evaluation Parameters (can be consistent with training) ---
        eval_img_size = IMAGE_SIZE if 'IMAGE_SIZE' in globals() else 640
        eval_batch_size = BATCH_SIZE if 'BATCH_SIZE' in globals() else 16 # Can often be larger for validation if memory allows
        eval_device = DEVICE if 'DEVICE' in globals() else ('0' if torch.cuda.is_available() else 'cpu')

        if 'yaml_file_path' not in globals() or not os.path.exists(yaml_file_path):
            logging.error(f"Dataset YAML configuration file not found at expected path: {globals().get('yaml_file_path', 'Path not defined')}.")
            logging.error("Cannot proceed with evaluation.")
            val_metrics = None
            test_metrics = None
        else:
            dataset_yaml_path_for_eval = yaml_file_path
            logging.info(f"Using dataset YAML for evaluation: {dataset_yaml_path_for_eval}")
            logging.info(f"Evaluation parameters: Image Size={eval_img_size}, Batch Size={eval_batch_size}, Device={eval_device}")

            # --- Evaluate on the Validation Set ---
            logging.info("Starting evaluation on the validation set...")
            try:
                val_results = model.val(
                    data=dataset_yaml_path_for_eval,
                    split='val', # Specify the validation split
                    imgsz=eval_img_size,
                    batch=eval_batch_size,
                    device=eval_device,
                    project=os.path.join(PROJECT_DIR, RUN_NAME, 'evaluation'), # Save val results in a subfolder
                    name='val_set_eval'
                )
                logging.info("Validation set evaluation completed.")
                # val_results object contains various metrics. For segmentation, these are typically:
                # val_results.box.map    # mAP50-95 for box
                # val_results.box.map50  # mAP50 for box
                # val_results.seg.map    # mAP50-95 for mask
                # val_results.seg.map50  # mAP50 for mask
                # val_results.speed (preprocess, inference, postprocess speeds)
                # The metrics are also printed to console by model.val()
                # Storing the metrics directly if they are accessible as a dictionary or specific attributes
                # For now, model.val() prints them, and they are saved to its run directory.
                # We will collect them more formally in the next cell from the saved files or by re-parsing.
                # For simplicity in this cell, we acknowledge completion.
                # The actual metrics values from val_results might need specific accessors.
                # Ultralytics val_results.metrics often gives a dict.
                val_metrics = val_results.metrics.results_dict if hasattr(val_results, 'metrics') and hasattr(val_results.metrics, 'results_dict') else None
                if val_metrics:
                     logging.info(f"Validation metrics summary: {val_metrics}")
                else:
                     logging.warning("Could not directly access validation metrics dictionary from results object. Will rely on saved files for Cell 10.")


            except Exception as e_val:
                logging.error(f"An error occurred during validation set evaluation: {e_val}")
                val_metrics = None

            # --- Evaluate on the Test Set ---
            logging.info("Starting evaluation on the test set...")
            try:
                test_results = model.val(
                    data=dataset_yaml_path_for_eval,
                    split='test', # Specify the test split
                    imgsz=eval_img_size,
                    batch=eval_batch_size,
                    device=eval_device,
                    project=os.path.join(PROJECT_DIR, RUN_NAME, 'evaluation'), # Save test results
                    name='test_set_eval'
                )
                logging.info("Test set evaluation completed.")
                test_metrics = test_results.metrics.results_dict if hasattr(test_results, 'metrics') and hasattr(test_results.metrics, 'results_dict') else None
                if test_metrics:
                     logging.info(f"Test metrics summary: {test_metrics}")
                else:
                     logging.warning("Could not directly access test metrics dictionary from results object. Will rely on saved files for Cell 10.")

            except Exception as e_test:
                logging.error(f"An error occurred during test set evaluation: {e_test}")
                test_metrics = None
                
    except NameError as ne:
        logging.error(f"NameError: {ne}. YOLO class might not be imported.")
        val_metrics = None
        test_metrics = None
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        val_metrics = None
        test_metrics = None

logging.info("Cell 9 execution completed: Model loading and evaluation on validation/test sets attempted.")

2025-05-13 01:29:39,725 - INFO - Starting Cell 9: Loading best trained model and evaluating.
2025-05-13 01:29:39,726 - INFO - Path to the best model weights: ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt
2025-05-13 01:29:40,187 - INFO - Successfully loaded the best trained model from: ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt
2025-05-13 01:29:40,187 - INFO - Using dataset YAML for evaluation: ./ADE20K_YOLOv8_Dataset/ade20k_yolo.yaml
2025-05-13 01:29:40,188 - INFO - Evaluation parameters: Image Size=640, Batch Size=16, Device=0
2025-05-13 01:29:40,188 - INFO - Starting evaluation on the validation set...


Ultralytics 8.3.133 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,807,464 parameters, 0 gradients, 42.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2236.1±1031.5 MB/s, size: 216.0 KB)


val: Scanning /home/sagemaker-user/ADE20K_YOLOv8_Dataset/labels/val.cache... 2669 images, 15 backgrounds, 0 corrupt: 100%|██████████| 2669/2669 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/167 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   1%|          | 1/167 [00:00<01:25,  1.94it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 167/167 [00:41<00:00,  4.02it/s]


                   all       2670      18146      0.499      0.342       0.35       0.25      0.497      0.332      0.333      0.205
airplane, aeroplane, plane         25         40      0.645      0.575      0.602      0.405      0.565        0.5      0.531      0.365
backpack, back pack, knapsack, packsack, rucksack, haversack         25         35      0.333     0.0857      0.103     0.0663      0.343     0.0857      0.106      0.068
                   bed        227        262      0.784      0.889      0.908      0.811      0.797      0.901      0.909      0.696
                 bench         76        115      0.385       0.33       0.28      0.206       0.37      0.313      0.266      0.155
bicycle, bike, wheel, cycle         36         49       0.45      0.551        0.5      0.287      0.397      0.483      0.382      0.133
                  bird         10         19      0.175     0.0526     0.0865     0.0422      0.181     0.0526      0.117     0.0366
               blender

/opt/conda/lib/python3.12/site-packages/ultralytics/utils/metrics.py:450: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.12/site-packages/ultralytics/utils/metrics.py:450: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.savefig(plot_fname, dpi=250)


Speed: 0.3ms preprocess, 8.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/evaluation/test_set_eval


2025-05-13 01:31:21,393 - INFO - Test set evaluation completed.
2025-05-13 01:31:21,402 - WARNING - Could not directly access test metrics dictionary from results object. Will rely on saved files for Cell 10.
2025-05-13 01:31:21,403 - INFO - Cell 9 execution completed: Model loading and evaluation on validation/test sets attempted.


In [ ]:
# Cell 10: Calculate and Compile Metrics

import pandas as pd
import numpy as np # For F1 score calculation if P+R is zero
import logging
import os

# Standard basic logging setup.
# logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s') # Assume configured

logging.info("Starting Cell 10: Calculating and Compiling Metrics.")

# Initialize a list to store dictionaries of metrics for each set
compiled_metrics_data = []

# --- Helper function to calculate F1 score ---
def calculate_f1_score(precision, recall):
    """Calculates F1 score, handling cases where precision or recall might be zero."""
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

# --- 1. Metrics from Training (Validation at Best Epoch) ---
best_training_epoch = 88

# Construct the correct path to results.csv using PROJECT_DIR and RUN_NAME from Cell 8
# Ensure PROJECT_DIR and RUN_NAME are available in the global scope from Cell 8.
if 'PROJECT_DIR' not in globals() or 'RUN_NAME' not in globals():
    logging.error("PROJECT_DIR or RUN_NAME from Cell 8 not found. Cannot construct path to results.csv.")
    training_results_csv_path = None # Path cannot be formed
else:
    training_results_csv_path = os.path.join(PROJECT_DIR, RUN_NAME, 'results.csv')
    logging.info(f"Attempting to load training results from: {training_results_csv_path}")


if training_results_csv_path and os.path.exists(training_results_csv_path):
    try:
        train_results_df = pd.read_csv(training_results_csv_path)
        train_results_df.columns = train_results_df.columns.str.strip()
        
        # Get the row for the best epoch (epoch numbers in CSV are 1-indexed)
        best_epoch_metrics_row = train_results_df[train_results_df['epoch'] == best_training_epoch]

        if not best_epoch_metrics_row.empty:
            box_p_train_val = best_epoch_metrics_row['metrics/precision(B)'].iloc[0]
            box_r_train_val = best_epoch_metrics_row['metrics/recall(B)'].iloc[0]
            box_map50_train_val = best_epoch_metrics_row['metrics/mAP50(B)'].iloc[0]
            box_map50_95_train_val = best_epoch_metrics_row['metrics/mAP50-95(B)'].iloc[0]
            box_f1_train_val = calculate_f1_score(box_p_train_val, box_r_train_val)

            mask_p_train_val = best_epoch_metrics_row['metrics/precision(M)'].iloc[0]
            mask_r_train_val = best_epoch_metrics_row['metrics/recall(M)'].iloc[0]
            mask_map50_train_val = best_epoch_metrics_row['metrics/mAP50(M)'].iloc[0]
            mask_map50_95_train_val = best_epoch_metrics_row['metrics/mAP50-95(M)'].iloc[0]
            mask_f1_train_val = calculate_f1_score(mask_p_train_val, mask_r_train_val)
            
            compiled_metrics_data.append({
                'Set': f'Validation (during Training Epoch {best_training_epoch})',
                'Box_Precision': round(box_p_train_val, 4),
                'Box_Recall': round(box_r_train_val, 4),
                'Box_mAP50': round(box_map50_train_val, 4),
                'Box_mAP50-95': round(box_map50_95_train_val, 4),
                'Box_F1': round(box_f1_train_val, 4),
                'Mask_Precision': round(mask_p_train_val, 4),
                'Mask_Recall': round(mask_r_train_val, 4),
                'Mask_mAP50': round(mask_map50_train_val, 4),
                'Mask_mAP50-95': round(mask_map50_95_train_val, 4),
                'Mask_F1': round(mask_f1_train_val, 4)
            })
            logging.info(f"Successfully extracted metrics for Validation (during Training Epoch {best_training_epoch}).")
        else:
            logging.warning(f"Could not find data for epoch {best_training_epoch} in {training_results_csv_path}.")
    except FileNotFoundError: # This specific exception check is good
        logging.error(f"Training results file '{training_results_csv_path}' not found. Cannot compile training validation metrics.")
    except Exception as e:
        logging.error(f"Error processing {training_results_csv_path}: {e}")
elif training_results_csv_path: # Path was formed but file doesn't exist
    logging.error(f"Training results CSV file not found at the constructed path: {training_results_csv_path}")
else: # PROJECT_DIR or RUN_NAME was missing
     logging.error("Path to training results CSV could not be constructed due to missing PROJECT_DIR or RUN_NAME variables.")


# --- 2. Metrics from Dedicated Validation Set Evaluation (Cell 9 Console Output) ---
try:
    val_box_p = 0.503
    val_box_r = 0.37
    val_box_map50 = 0.368
    val_box_map50_95 = 0.260 
    val_box_f1 = calculate_f1_score(val_box_p, val_box_r)

    val_mask_p = 0.517
    val_mask_r = 0.342
    val_mask_map50 = 0.347
    val_mask_map50_95 = 0.216
    val_mask_f1 = calculate_f1_score(val_mask_p, val_mask_r)

    compiled_metrics_data.append({
        'Set': 'Validation (Dedicated Eval)',
        'Box_Precision': round(val_box_p, 4),
        'Box_Recall': round(val_box_r, 4),
        'Box_mAP50': round(val_box_map50, 4),
        'Box_mAP50-95': round(val_box_map50_95, 4),
        'Box_F1': round(val_box_f1, 4),
        'Mask_Precision': round(val_mask_p, 4),
        'Mask_Recall': round(val_mask_r, 4),
        'Mask_mAP50': round(val_mask_map50, 4),
        'Mask_mAP50-95': round(val_mask_map50_95, 4),
        'Mask_F1': round(val_mask_f1, 4)
    })
    logging.info("Successfully compiled metrics for Dedicated Validation Set Evaluation.")
except Exception as e:
    logging.error(f"Error inputting dedicated validation metrics: {e}")


# --- 3. Metrics from Dedicated Test Set Evaluation (Cell 9 Console Output) ---
# From user log: all 2670 18146 0.479 0.348 0.348 0.247 0.472 0.33 0.328 0.202
try:
    test_box_p = 0.479
    test_box_r = 0.348
    test_box_map50 = 0.348
    test_box_map50_95 = 0.247
    test_box_f1 = calculate_f1_score(test_box_p, test_box_r)

    test_mask_p = 0.472
    test_mask_r = 0.330 
    test_mask_map50 = 0.328
    test_mask_map50_95 = 0.202
    test_mask_f1 = calculate_f1_score(test_mask_p, test_mask_r)

    compiled_metrics_data.append({
        'Set': 'Test (Dedicated Eval)',
        'Box_Precision': round(test_box_p, 4),
        'Box_Recall': round(test_box_r, 4),
        'Box_mAP50': round(test_box_map50, 4),
        'Box_mAP50-95': round(test_box_map50_95, 4),
        'Box_F1': round(test_box_f1, 4),
        'Mask_Precision': round(test_mask_p, 4),
        'Mask_Recall': round(test_mask_r, 4),
        'Mask_mAP50': round(test_mask_map50, 4),
        'Mask_mAP50-95': round(test_mask_map50_95, 4),
        'Mask_F1': round(test_mask_f1, 4)
    })
    logging.info("Successfully compiled metrics for Dedicated Test Set Evaluation.")
except Exception as e:
    logging.error(f"Error inputting dedicated test metrics: {e}")


if compiled_metrics_data:
    logging.info("\n--- Compiled Metrics Summary ---")
    temp_df = pd.DataFrame(compiled_metrics_data)
    # Log dataframe string representation for easier viewing in logs
    df_string_summary = temp_df.to_string()
    for line in df_string_summary.split('\n'):
        logging.info(line)
else:
    logging.warning("No metrics were compiled. compiled_metrics_data is empty.")

logging.info("Cell 10 execution completed: Metrics have been calculated and compiled.")

2025-05-13 01:31:25,162 - INFO - Starting Cell 10: Calculating and Compiling Metrics.
2025-05-13 01:31:25,163 - INFO - Attempting to load training results from: ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/results.csv
2025-05-13 01:31:25,168 - INFO - Successfully extracted metrics for Validation (during Training Epoch 88).
2025-05-13 01:31:25,169 - INFO - Successfully compiled metrics for Dedicated Validation Set Evaluation.
2025-05-13 01:31:25,170 - INFO - Successfully compiled metrics for Dedicated Test Set Evaluation.
2025-05-13 01:31:25,171 - INFO - 
--- Compiled Metrics Summary ---
2025-05-13 01:31:25,177 - INFO -                                      Set  Box_Precision  Box_Recall  Box_mAP50  Box_mAP50-95  Box_F1  Mask_Precision  Mask_Recall  Mask_mAP50  Mask_mAP50-95  Mask_F1
2025-05-13 01:31:25,177 - INFO - 0  Validation (during Training Epoch 88)         0.5296      0.3675     0.3732        0.2659  0.4339          0.5186       0.3586      0.3544         0.2209   0.424

In [9]:
# Cell 11: Locate and List Generated Visualizations

import os
import logging

# Ensure PROJECT_DIR and RUN_NAME (from Cell 8) are available.

logging.info("Starting Cell 11: Locating and Listing Generated Visualizations.")

# Base directory for the main training run
main_run_dir = ""
if 'PROJECT_DIR' in globals() and 'RUN_NAME' in globals():
    main_run_dir = os.path.join(PROJECT_DIR, RUN_NAME)
    logging.info(f"Main training run directory: {os.path.abspath(main_run_dir)}")
else:
    logging.error("PROJECT_DIR or RUN_NAME not defined. Cannot locate visualization files.")

# Base directories for the dedicated evaluation runs (from Cell 9)
val_eval_dir = ""
if main_run_dir: # evaluation dir is sub-dir of main_run_dir
    val_eval_dir = os.path.join(main_run_dir, 'evaluation', 'val_set_eval')
    logging.info(f"Dedicated validation evaluation directory: {os.path.abspath(val_eval_dir)}")

test_eval_dir = ""
if main_run_dir:
    test_eval_dir = os.path.join(main_run_dir, 'evaluation', 'test_set_eval')
    logging.info(f"Dedicated test evaluation directory: {os.path.abspath(test_eval_dir)}")

visualization_paths = {}

if main_run_dir:
    # --- Training History and Key Plots from Main Training Run ---
    # These plots are generated by YOLOv8 during the model.train() process.
    
    # Training history (losses, mAP vs. epoch)
    results_png_path = os.path.join(main_run_dir, 'results.png')
    if os.path.exists(results_png_path):
        visualization_paths['Training History (results.png)'] = os.path.abspath(results_png_path)
    else:
        logging.warning(f"'results.png' not found in {main_run_dir}")

    # Precision-Recall curve from training
    pr_curve_train_path = os.path.join(main_run_dir, 'PR_curve.png')
    if os.path.exists(pr_curve_train_path):
        visualization_paths['PR Curve (Training Validation)'] = os.path.abspath(pr_curve_train_path)
    else:
        logging.warning(f"'PR_curve.png' not found in {main_run_dir}")

    # Confusion matrix from training
    confusion_matrix_train_path = os.path.join(main_run_dir, 'confusion_matrix.png')
    if os.path.exists(confusion_matrix_train_path):
        visualization_paths['Confusion Matrix (Training Validation)'] = os.path.abspath(confusion_matrix_train_path)
    else:
        logging.warning(f"'confusion_matrix.png' not found in {main_run_dir}")
        
    # Labels correlogram
    labels_correlogram_path = os.path.join(main_run_dir, 'labels_correlogram.jpg')
    if os.path.exists(labels_correlogram_path):
        visualization_paths['Labels Correlogram'] = os.path.abspath(labels_correlogram_path)
    else:
        logging.warning(f"'labels_correlogram.jpg' not found in {main_run_dir}")

    # Labels distribution
    labels_path = os.path.join(main_run_dir, 'labels.jpg')
    if os.path.exists(labels_path):
        visualization_paths['Labels Distribution'] = os.path.abspath(labels_path)
    else:
        logging.warning(f"'labels.jpg' not found in {main_run_dir}")


    # --- Plots from Dedicated Validation Set Evaluation (Cell 9) ---
    # model.val() also generates plots in its specified directory.
    if val_eval_dir and os.path.exists(val_eval_dir):
        val_pr_curve_path = os.path.join(val_eval_dir, 'PR_curve.png')
        if os.path.exists(val_pr_curve_path):
            visualization_paths['PR Curve (Dedicated Validation Eval)'] = os.path.abspath(val_pr_curve_path)
        else:
            logging.warning(f"'PR_curve.png' not found in dedicated validation eval directory: {val_eval_dir}")
        
        val_confusion_matrix_path = os.path.join(val_eval_dir, 'confusion_matrix.png')
        if os.path.exists(val_confusion_matrix_path):
            visualization_paths['Confusion Matrix (Dedicated Validation Eval)'] = os.path.abspath(val_confusion_matrix_path)
        else:
            logging.warning(f"'confusion_matrix.png' not found in dedicated validation eval directory: {val_eval_dir}")

    # --- Plots from Dedicated Test Set Evaluation (Cell 9) ---
    if test_eval_dir and os.path.exists(test_eval_dir):
        test_pr_curve_path = os.path.join(test_eval_dir, 'PR_curve.png')
        if os.path.exists(test_pr_curve_path):
            visualization_paths['PR Curve (Dedicated Test Eval)'] = os.path.abspath(test_pr_curve_path)
        else:
            logging.warning(f"'PR_curve.png' not found in dedicated test eval directory: {test_eval_dir}")

        test_confusion_matrix_path = os.path.join(test_eval_dir, 'confusion_matrix.png')
        if os.path.exists(test_confusion_matrix_path):
            visualization_paths['Confusion Matrix (Dedicated Test Eval)'] = os.path.abspath(test_confusion_matrix_path)
        else:
            logging.warning(f"'confusion_matrix.png' not found in dedicated test eval directory: {test_eval_dir}")
            
    if visualization_paths:
        logging.info("\n--- Key Visualization Files ---")
        for name, path in visualization_paths.items():
            logging.info(f"  {name}: {path}")
    else:
        logging.warning("No visualization files were explicitly found or PROJECT_DIR/RUN_NAME were not set. "
                        "Please check the YOLOv8 run directories manually if they were generated.")

    logging.info("\nNote on ROC-AUC Curves:")
    logging.info("YOLOv8's standard outputs for detection/segmentation primarily include Precision-Recall (PR) curves "
                 "and confusion matrices. While components for ROC curves exist within the data, "
                 "generating a plottable ROC-AUC curve image file per class or overall is not a default artifact "
                 "and would typically require custom plotting code based on prediction scores and ground truths.")

else:
    logging.error("PROJECT_DIR or RUN_NAME not available. Cannot list visualization paths.")


logging.info("Cell 11 execution completed: Paths to generated visualizations have been listed.")

2025-05-13 01:31:31,754 - INFO - Starting Cell 11: Locating and Listing Generated Visualizations.
2025-05-13 01:31:31,756 - INFO - Main training run directory: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12
2025-05-13 01:31:31,757 - INFO - Dedicated validation evaluation directory: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/evaluation/val_set_eval
2025-05-13 01:31:31,758 - INFO - Dedicated test evaluation directory: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/evaluation/test_set_eval
2025-05-13 01:31:31,759 - WARNING - 'results.png' not found in ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12
2025-05-13 01:31:31,759 - WARNING - 'PR_curve.png' not found in ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12
2025-05-13 01:31:31,760 - WARNING - 'confusion_matrix.png' not found in ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12
2025-05-13 01:31:31,761 - WARNING - 'PR_curve.png' not foun

In [10]:
# Cell 12: Save Compiled Metrics to CSV File

import pandas as pd
import os
import logging

# Ensure compiled_metrics_data (list of dictionaries from Cell 10) is available.
# Ensure PROJECT_DIR and RUN_NAME (from Cell 8) are available for referencing other artifacts.
# Ensure visualization_paths (from Cell 11) is available.

logging.info("Starting Cell 12: Saving Compiled Metrics to CSV File.")

# Define the name and path for the output CSV file.
# This will be saved in the current working directory of the notebook.
output_metrics_csv_filename = "ThirdEye_model_evaluation_metrics.csv"
output_metrics_csv_path = os.path.join('.', output_metrics_csv_filename) # Save in the notebook's root

# Path to the detailed training history CSV (results.csv from YOLOv8 run)
# This was defined and used in Cell 10 (Amended)
training_history_csv_path = ""
if 'PROJECT_DIR' in globals() and 'RUN_NAME' in globals():
    training_history_csv_path = os.path.abspath(os.path.join(PROJECT_DIR, RUN_NAME, 'results.csv'))
else:
    logging.warning("PROJECT_DIR or RUN_NAME not defined; cannot provide full path for training history CSV.")


if 'compiled_metrics_data' in globals() and compiled_metrics_data:
    try:
        # Convert the list of dictionaries into a Pandas DataFrame
        metrics_df = pd.DataFrame(compiled_metrics_data)

        # Save the DataFrame to a CSV file
        metrics_df.to_csv(output_metrics_csv_path, index=False)
        logging.info(f"Successfully saved compiled model metrics to: {os.path.abspath(output_metrics_csv_path)}")

        # --- Log information about other artifacts ---
        logging.info("\n--- Summary of Key Output Artifacts ---")
        logging.info(f"1. Compiled Numerical Metrics: {os.path.abspath(output_metrics_csv_path)}")
        
        if training_history_csv_path and os.path.exists(training_history_csv_path):
            logging.info(f"2. Detailed Training History (epoch-wise data): {training_history_csv_path}")
        else:
            logging.warning("Detailed training history file (results.csv) path could not be confirmed or file not found.")

        logging.info("3. Visualization Files (refer to Cell 11 output for specific paths if found):")
        if 'visualization_paths' in globals() and visualization_paths:
            for name, path in visualization_paths.items():
                if os.path.exists(path):
                    logging.info(f"   - {name}: {path}")
                else:
                    logging.warning(f"   - {name}: File previously noted at {path} was not found during this check.")
            if not any('PR Curve' in name for name in visualization_paths.keys()):
                 logging.warning("   - PR Curves: As noted in Cell 11, specific PR_curve.png files were not found in expected locations. "
                                 "The Precision and Recall values are in the numerical metrics CSV. "
                                 "The 'results.png' contains PR curve trends over epochs.")
        else:
            logging.warning("   - Visualization paths dictionary not found or empty from Cell 11.")
            logging.info("     Please check the training run directory for plots: "
                         f"{os.path.abspath(os.path.join(PROJECT_DIR, RUN_NAME)) if 'PROJECT_DIR' in globals() else 'PROJECT_DIR_NOT_SET'}")

        logging.info("\nNote on ROC-AUC Curves:")
        logging.info("   As mentioned previously, dedicated ROC-AUC curve image files are not a standard default output "
                     "from YOLOv8 training/evaluation for detection/segmentation tasks.")

    except Exception as e:
        logging.error(f"An error occurred while saving metrics to CSV or logging artifact paths: {e}")
else:
    logging.warning("Global variable 'compiled_metrics_data' not found or is empty. Cannot save metrics to CSV.")


logging.info("Cell 12 execution completed: Attempted to save metrics and list artifact locations.")

2025-05-13 01:31:37,248 - INFO - Starting Cell 12: Saving Compiled Metrics to CSV File.
2025-05-13 01:31:37,252 - INFO - Successfully saved compiled model metrics to: /home/sagemaker-user/ThirdEye_model_evaluation_metrics.csv
2025-05-13 01:31:37,253 - INFO - 
--- Summary of Key Output Artifacts ---
2025-05-13 01:31:37,254 - INFO - 1. Compiled Numerical Metrics: /home/sagemaker-user/ThirdEye_model_evaluation_metrics.csv
2025-05-13 01:31:37,254 - INFO - 2. Detailed Training History (epoch-wise data): /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/results.csv
2025-05-13 01:31:37,255 - INFO - 3. Visualization Files (refer to Cell 11 output for specific paths if found):
2025-05-13 01:31:37,256 - INFO -    - Labels Correlogram: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/labels_correlogram.jpg
2025-05-13 01:31:37,256 - INFO -    - Labels Distribution: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/labels.jpg


In [11]:
# Cell 13: Export Best Trained Model to ONNX Format

import os
import logging
import torch # To check device, though export often specifies its own device handling

# Ensure YOLO class is imported (from Cell 7)
# Ensure PROJECT_DIR, RUN_NAME (from Cell 8) are available to locate best.pt
# Ensure IMAGE_SIZE (from Cell 8) is available for specifying export image size.

logging.info("Starting Cell 13: Exporting Best Trained Model to ONNX Format.")

# Path to the best trained model weights (.pt file)
best_model_pt_path = ""
if 'PROJECT_DIR' in globals() and 'RUN_NAME' in globals():
    best_model_pt_path = os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'best.pt')
    logging.info(f"Path to the best .pt model weights: {best_model_pt_path}")
else:
    logging.error("PROJECT_DIR or RUN_NAME not defined. Cannot locate the .pt model for ONNX export.")
    # Exit or prevent further execution if path is not found

if best_model_pt_path and os.path.exists(best_model_pt_path):
    try:
        # Load the best .pt model
        # The YOLO object needs to be initialized with the .pt file to load the weights.
        model = YOLO(best_model_pt_path)
        logging.info(f"Successfully loaded model from {best_model_pt_path} for ONNX export.")

        # Define image size for export (should ideally match training imgsz)
        export_img_size = IMAGE_SIZE if 'IMAGE_SIZE' in globals() else 640
        
        # Define the desired path for the ONNX model.
        # By default, export() saves it in the same directory as the .pt file.
        # e.g., 'path/to/weights/best.onnx'
        # We can let it save there, or specify a different 'save_dir' or 'name' via export args.
        # For simplicity, we'll let it use the default location and then log that path.
        
        logging.info(f"Starting ONNX export for model: {best_model_pt_path} with image size: {export_img_size}x{export_img_size}")

        # Export the model to ONNX format
        # The export function may have additional arguments like opset, dynamic_axes, etc.
        # For now, a basic export is performed.
        # The returned path is the path to the exported ONNX model.
        onnx_model_path = model.export(
            format='onnx',
            imgsz=export_img_size,
            # opset=12, # Optional: specify ONNX opset version if needed, default is usually fine.
            # dynamic=False # Optional: set to True for dynamic input/output axes.
        )
        # model.export() saves the file (e.g., yolov8s-seg.onnx) typically in the same dir as the .pt model
        # or in a new directory if 'project' and 'name' are specified in export.
        # The returned path from model.export() is the actual path it was saved to.

        logging.info(f"Model successfully exported to ONNX format at: {os.path.abspath(onnx_model_path)}")

    except NameError as ne:
        logging.error(f"NameError: {ne}. This might indicate that the YOLO class or other necessary variables "
                      "like IMAGE_SIZE, PROJECT_DIR, RUN_NAME were not available.")
    except FileNotFoundError as fnfe:
        logging.error(f"FileNotFoundError during export: {fnfe}. Ensure the .pt model file exists at the specified path.")
    except Exception as e:
        logging.error(f"An unexpected error occurred during ONNX export: {e}")
        # This can sometimes be due to environment issues or specific model ops not supported by the default ONNX opset.
        # Checking the detailed error message from Ultralytics (if any in console) would be helpful.
else:
    if best_model_pt_path : # If path was formed but file doesn't exist
        logging.error(f"Best .pt model file not found at {best_model_pt_path}. Cannot export to ONNX.")
    # else: PROJECT_DIR/RUN_NAME was missing, already logged.

logging.info("Cell 13 execution completed: Attempted to export model to ONNX format.")

2025-05-13 01:31:55,142 - INFO - Starting Cell 13: Exporting Best Trained Model to ONNX Format.
2025-05-13 01:31:55,143 - INFO - Path to the best .pt model weights: ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt
2025-05-13 01:31:55,218 - INFO - Successfully loaded model from ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt for ONNX export.
2025-05-13 01:31:55,219 - INFO - Starting ONNX export for model: ./YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt with image size: 640x640


Ultralytics 8.3.133 🚀 Python-3.12.9 torch-2.5.1 CPU (Intel Xeon Platinum 8259CL 2.50GHz)
YOLOv8s-seg summary (fused): 85 layers, 11,807,464 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 108, 8400), (1, 32, 160, 160)) (45.4 MB)

ONNX: starting export with onnx 1.18.0 opset 19...
ONNX: slimming with onnxslim 0.1.52...
ONNX: export success ✅ 2.0s, saved as 'YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.onnx' (45.3 MB)

Export complete (3.1s)
Results saved to /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights
Predict:         yolo predict task=segment model=YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.onnx imgsz=640  
Validate:        yolo val task=segment model=YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.onnx imgsz=640 data=./ADE20K_YOLOv8_Dataset/a

2025-05-13 01:31:58,275 - INFO - Model successfully exported to ONNX format at: /home/sagemaker-user/YOLOv8_Training_Runs_ADE20K/ade20k_yolov8s_seg_run12/weights/best.onnx
2025-05-13 01:31:58,275 - INFO - Cell 13 execution completed: Attempted to export model to ONNX format.
